## Ripple stats during sleep deprivation

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from ModulesPath.plotUtil import Fig
import subjects
import seaborn as sns

# sessions = subjects.Sd().allsess
sessions = subjects.Nsd().allsess
print(sessions)

[processData(RatJDay2), processData(RatKDay2), processData(RatNDay2), processData(RatSDay2NSD)]


In [5]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(grid=(1,1),size=(4,4),style='Pres')
df = pd.DataFrame()
for sub,sess in enumerate(sessions):

    post = sess.paradigm.post
    # pre = sess.paradigm.pre
    # sd_period = [post[0],post[1]]
    sd_period = [post[0],post[0]+9*3600]
    rpls = sess.ripple.epochs
    # states = sess.brainstates.epochs

    binsz = 600
    sd_bin = np.arange(sd_period[0],sd_period[1]+binsz,binsz)
    hist_rpl = np.histogram(rpls.start,bins=sd_bin)[0]

    nrem_percent = []
    for start in sd_bin[:-1]:
        prop = sess.brainstates.proportion([start,start+binsz])
        nrem_percent.append(prop['nrem'])

    df = df.append(pd.DataFrame({'n_ripples':hist_rpl/binsz,'tbin':(sd_bin[:-1]-sd_bin[0])/3600,'nrem':nrem_percent,'sub':sub}))


ax = plt.subplot(gs[0])
rpls_mean = df.groupby('tbin').mean().n_ripples
rpls_sem = df.groupby('tbin').sem(ddof=0).n_ripples
ax.fill_between(rpls_mean.index,rpls_mean-rpls_sem,rpls_mean+rpls_sem,color='#faa49d',alpha=0.7)
ax.plot(rpls_mean.index,rpls_mean,color='#faa49d',zorder=2)
ax.axvline(5,ls='--',color='r')
ax.set_xlabel('ZT time (h)')
ax.set_ylabel('Ripple rate')

axsleep= ax.twinx()
color = sess.brainstates.colors['nrem']
sleep_df = df.groupby('tbin').mean()
axsleep.fill_between(sleep_df.index,sleep_df.nrem,color=color,zorder=1,alpha=0.3)
# sns.lineplot(data=df,x='tbin',y='nrem',ax=axsleep,estimator='sum')
axsleep.set_ylim([0,1])
axsleep.spines['right'].set_visible(True)
axsleep.set_ylabel('nrem proportion')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'nrem proportion')

In [132]:
ep = sess.brainstates.epochs
period = [start,start+binsz]
ep = ep[(ep.stop>period[0])&(ep.start<period[1])].reset_index(drop=True)
if (f:=ep['start'].iloc[0]) < period[0]:
   ep.at[0,'start']= period[0]

if (f:=ep['stop'].iloc[-1]) > period[1]:
    ep.at[ep.index[-1],'stop'] = period[1]

ep_group = ep.groupby('label').sum().duration/np.diff(period)
ep_group.index.values

states_proportion = {'rem':0,'nrem':0,'quiet':0,'active':0}

for state in ep_group.index.values:
    states_proportion[state] = ep_group[state]

ep



,start,stop,duration,label
0,42485.0,42574.1,225.6,rem
1,42574.1,42601.3,27.2,active
2,42601.3,42670.1,68.8,quiet
3,42670.1,42819.7,149.6,nrem
4,42819.7,43075.7,256.0,quiet
5,43075.7,43085.0,77.6,nrem


In [4]:
import numpy as np
import subjects
from ModulesPath.getPosition import SessPosition,SessTrack

sess = subjects.Sd().ratNday1[0]

position = SessPosition

